In [3]:
import bs4
import pyodbc
import time
import datetime as dt
from urllib.request import urlopen as uo
from bs4 import BeautifulSoup as soup

In [4]:
base_url = "https://kabarmedan.com"
berita_medan = "/feed/"

In [5]:
def getNewsList():
    url = "https://sumutpos.co/feed/"
    uClient = uo(url)
    list_page = uClient.read()
    uClient.close()

    page_soup = soup(list_page, "xml")
    items = page_soup.findAll("item")
    for i in items:
        news_link = i.find("link").text
        getNewsData(news_link)

In [13]:
def getNewsData(news_link):
    uClient = uo(news_link)
    news_page = uClient.read()
    uClient.close()
    
    news_article = ""
    counter = 0
    page_soup = soup(news_page, "html.parser")
    news_card = page_soup.find("div", {"class":"col-xs-12 c-single"})
    news_title = news_card.find("div", {"class":"ol-content-title-single"}).text.replace("\t","").replace("\n","")
    news_category = page_soup.findAll("li", {"class":"breadcumb-link"})[1].text
    news_author = None
    news_editor = None
    news_postedAt = news_card.find("li", {"class":"meta-single"}).findAll("strong")[0].text
    news_article_raw = page_soup.findAll("p")
    for i in news_article_raw:
        if len(i.text) > 50:
            news_article = news_article + i.text.replace("no picture","")
            if "SUMUTPOS.CO" in i.text:
                news_location = i.text.split(", ")[0].title()
    news_crawledAt = dt.datetime.now()
    news_image = news_card.find("img")["data-permalink"]
    news_image_detail = page_soup.findAll("p", {"class":"wp-caption-text"})[1].text
    if news_image_detail in news_article:
        news_article = news_article.replace(news_image_detail,"").strip()
    news_id1 = news_link.split("/")[6]
    news_id2 = news_id1.split("-")[0]
    news_id3 = news_id2[:5]
    news_id4 = news_link.split("/")[5]
    news_id5 = len(news_link)
    news_id6 = str(news_id5)
    news_id = news_id6+news_id3+news_id4
    news_tags = page_soup.find("div", {"class":"text-tags"}).text.split(" :")[1].replace("#",", ").strip()
#     print(news_title)
#     print(news_category)
#     print(news_author)
#     print(news_postedAt)
#     print(news_location)
#     print(news_article)
#     print(news_crawledAt)
#     print(news_image)
#     print(news_id)
#     print(news_editor)
#     print(news_source)
#     print(news_tags)


    trigger = 0
    trigger_tag = 0

    c.execute("SELECT * FROM news_portal_sumutpos WHERE news_id=?",news_id)
    for row in c:
        if row.news_id is not None:
            trigger = 1
            
    if trigger == 0:
        c.execute("""INSERT INTO news_portal_sumutpos (news_id, news_category, news_header, news_author, news_editor, news_timestamp, 
                    news_article, news_url, news_created, news_location) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", 
                  (news_id, news_category, news_title, news_author, news_editor, news_postedAt, 
                    news_article, news_link, news_crawledAt, news_location))
        conn.commit()
        print(news_id)
    else:
        print("Data with that ID already existed.")
    trigger = 0
    
    c.execute("SELECT * FROM news_portal_sumutpos_tag WHERE news_id=?",news_id)
    for row in c:
        if row.news_id is not None:
            trigger_tag = 1
            
    if trigger_tag == 0:
        c.execute("""INSERT INTO news_portal_sumutpos_tag (news_id, news_tag, news_created, tag_url) VALUES (?, ?, ?, ?)""", 
                  (news_id, news_tags, news_created, tag_url))
        conn.commit()
    else:
        print("Tag with that ID already existed.")
    trigger_tag = 0


In [9]:
def main():
    getNewsList()

In [14]:
if __name__ == "__main__":
    main()

In [10]:
news_link = "https://sumutpos.co/2019/03/04/ditangkap-di-kamar-hotel-andi-arief-berstatus-terperiksa/"
uClient = uo(news_link)
news_page = uClient.read()
uClient.close()

news_article = ""
counter = 0
page_soup = soup(news_page, "html.parser")
news_card = page_soup.find("div", {"class":"col-xs-12 c-single"})
news_title = news_card.find("div", {"class":"ol-content-title-single"}).text.replace("\t","").replace("\n","")
news_category = page_soup.findAll("li", {"class":"breadcumb-link"})[1].text
news_author = None
news_editor = None
news_source = None
news_postedAt = news_card.find("li", {"class":"meta-single"}).findAll("strong")[0].text
news_article_raw = page_soup.findAll("p")[1:]
for i in news_article_raw:
    if len(i.text) > 150:
        news_article = news_article + i.text.replace("no picture","")
        if "SUMUTPOS.CO" in i.text:
            news_location = i.find("strong").text.split(", ")[0].title()
news_crawledAt = dt.datetime.now()
news_image = news_card.find("img")["data-permalink"]
news_image_detail = page_soup.findAll("p", {"class":"wp-caption-text"})[1].text.strip()
if news_image_detail in news_article:
    news_article = news_article.replace(news_image_detail,"").strip()
news_id1 = news_link.split("/")[6]
news_id2 = news_id1.split("-")[0]
news_id3 = news_id2[:5]
news_id4 = news_link.split("/")[5]
news_id5 = len(news_link)
news_id6 = str(news_id5)
news_id = news_id6+news_id3+news_id4


news_tags = page_soup.find("div", {"class":"text-tags"}).text.split(" :")[1].replace("#",", ").strip()
test = page_soup.findAll("p", {"class":"wp-caption-text"})[1].text


print(news_title)
print(news_category)
print(news_author)
print(news_postedAt)
print(news_location)
print(news_id)
print(news_article)
print(news_crawledAt)
print(news_image)
print(news_tags)

Ditangkap di Kamar Hotel, Andi Arief Berstatus Terperiksa
Berita Foto2, Hukum & Kriminal, Nasional
None
Mar 4, 2019
Jakarta
88ditan04
JAKARTA, SUMUTPOS.CO – Kepala Divisi Humas Mabes Polri Inspektur Jenderal M. Iqbal mengungkapkan bahwa politisi Demokrat Andi Arief berstatus terperiksa. Sebelumnya, Andi Arief ditangkap di Hotel Peninsula, Jakarta Barat, Minggu (3/3/2019) malam, karena kasus dugaan penggunaan narkoba. Iqbal mengatakan aparat kepolisian memiliki waktu 3 x 24 jam untuk menentukan status Andi Arief.“Ya kan kita ada mekanisme, ada lex spesialis, di dalam proses penegakan hukum di narkoba ini. 3 x 24 jam,” ungkap Iqbal usai jumpa pers di Mabes Polri, Jakarta, Senin (4/3/2019) sore.Hingga saat ini, kepolisian masih menduga bahwa Wakil Sekjen Partai Demokrat Andi Arief sebagai pengguna narkoba jenis sabu. Aparat belum menemukan bukti bahwa Andi terlibat peredaran narkoba. “Belum ditemukan bukti-bukti, fakta-fakta kuat apakah saudara AA berkolerasi dengan kelompok mana, mafia m